In [231]:
import os
import pandas as pd
import numpy as np

In [232]:
df = pd.read_csv('data.csv', index_col = 'ID')

### Базовая оценка данных (только числовые колонки)

Проверка типов данных на соответствие

In [233]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 271116 entries, 1 to 135571
Data columns (total 14 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Name    271116 non-null  object 
 1   Sex     271116 non-null  object 
 2   Age     261642 non-null  float64
 3   Height  210945 non-null  float64
 4   Weight  208241 non-null  float64
 5   Team    271116 non-null  object 
 6   NOC     271116 non-null  object 
 7   Games   271116 non-null  object 
 8   Year    271116 non-null  int64  
 9   Season  271116 non-null  object 
 10  City    271116 non-null  object 
 11  Sport   271116 non-null  object 
 12  Event   271116 non-null  object 
 13  Medal   39783 non-null   object 
dtypes: float64(3), int64(1), object(10)
memory usage: 31.0+ MB


Формат соответствует данным

Для удобства создадим список числовых фичей

In [234]:
num_cols = df.select_dtypes(['float64', 'int64']).columns
num_cols = num_cols[:-1]
num_cols

Index(['Age', 'Height', 'Weight'], dtype='object')

Удалим дубликаты

In [235]:
df.drop_duplicates(inplace=True)

Посмотрим, нет ли выбросов

In [236]:
df[num_cols].describe()

,Age,Height,Weight
count,260416.000000,210917.000000,208204.000000
mean,25.454776,175.338953,70.701778
std,6.163869,10.518507,14.349027
min,10.000000,127.000000,25.000000
25%,21.000000,168.000000,60.000000
50%,24.000000,175.000000,70.000000
75%,28.000000,183.000000,79.000000
max,97.000000,226.000000,214.000000


Показатели в норме (оцениваем экстремальные значения)

In [237]:
df[num_cols].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 269731 entries, 1 to 135571
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Age     260416 non-null  float64
 1   Height  210917 non-null  float64
 2   Weight  208204 non-null  float64
dtypes: float64(3)
memory usage: 8.2 MB


In [238]:
df[num_cols].mean()

Age        25.454776
Height    175.338953
Weight     70.701778
dtype: float64

In [239]:
df[num_cols].median()

Age        24.0
Height    175.0
Weight     70.0
dtype: float64

In [240]:
df[num_cols].mode()

,Age,Height,Weight
0,23.0,180.0,70.0


### Заполнение пропусков

Начнём восстанавливать данные следующим образом:

    будем использовать среднее значение (у нас нет выбросов и среднее очень близко к медиане), группируя по колонке Event

In [241]:
from tqdm import tqdm

In [242]:
%%time
for col in tqdm(num_cols[:-1]):
    df[col] = df[['Event', col]].groupby('Event').transform(lambda x: x.fillna(x.mean()))

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:02<00:00,  1.46s/it]

Wall time: 2.92 s


In [243]:
df[num_cols].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 269731 entries, 1 to 135571
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Age     269589 non-null  float64
 1   Height  267911 non-null  float64
 2   Weight  208204 non-null  float64
dtypes: float64(3)
memory usage: 8.2 MB


Теперь продолжим восстановление по колонке Sport

In [244]:
%%time
for col in tqdm(num_cols[:-1]):
    df[col] = df[['Sport', col]].groupby('Sport').transform(lambda x: x.fillna(x.mean()))

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  3.90it/s]

Wall time: 515 ms


In [245]:
df[num_cols].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 269731 entries, 1 to 135571
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Age     269731 non-null  float64
 1   Height  269632 non-null  float64
 2   Weight  208204 non-null  float64
dtypes: float64(3)
memory usage: 8.2 MB


Восстановили возраст
Попробуем восстановить рост и вес

In [246]:
df['Age'] = df['Age'].astype(int)

In [247]:
%%time
for col in tqdm(num_cols[1:]):
    df[col] = df[['Name', col]].groupby('Name').transform(lambda x: x.fillna(x.mean()))

100%|███████████████████████████████████████████████████████████████████████████████████| 2/2 [07:47<00:00, 233.87s/it]

Wall time: 7min 47s


In [248]:
df[num_cols].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 269731 entries, 1 to 135571
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Age     269731 non-null  int32  
 1   Height  269643 non-null  float64
 2   Weight  208593 non-null  float64
dtypes: float64(2), int32(1)
memory usage: 7.2 MB


Сомнительное по эффективности решение - проделать то же самое с Именем, но мы получили немного больше данных

Оставшиеся пропуски заполним, группируя по возрасту

In [249]:
%%time
for col in tqdm(num_cols[1:]):
    df[col] = df[['Age', col]].groupby('Age').transform(lambda x: x.fillna(x.mean()))

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  4.11it/s]

Wall time: 489 ms


In [250]:
df[num_cols].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 269731 entries, 1 to 135571
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Age     269731 non-null  int32  
 1   Height  269731 non-null  float64
 2   Weight  269689 non-null  float64
dtypes: float64(2), int32(1)
memory usage: 7.2 MB


Восстановили рост, теперь будем группировать по нему

In [251]:
df['Height'] = df['Height'].astype(int)

In [252]:
%%time
df[col] = df[['Height', 'Weight']].groupby('Height').transform(lambda x: x.fillna(x.mean()))

Wall time: 265 ms


In [253]:
df[num_cols].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 269731 entries, 1 to 135571
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Age     269731 non-null  int32  
 1   Height  269731 non-null  int32  
 2   Weight  269731 non-null  float64
dtypes: float64(1), int32(2)
memory usage: 6.2 MB


In [254]:
df['Weight'] = df['Weight'].astype(int)

Заполнили все пропуски!